In [1]:
import pandas as pd
import numpy as np

In [8]:
data = pd.DataFrame(np.arange(6).reshape(2, 3),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'], name='number')
                   )
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [9]:
result = data.unstack()
result

number  state   
one     Ohio        0
        Colorado    3
two     Ohio        1
        Colorado    4
three   Ohio        2
        Colorado    5
dtype: int64

In [11]:
result.unstack()

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [12]:
result.unstack(0)

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [15]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [16]:
s1 = pd.Series([0, 1, 2, 3], index=list('abcd'))
s2 = pd.Series([4, 5, 6], index=list('cde'))
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [17]:
data2.unstack(1)

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [18]:
data2.unstack(1).stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [20]:
data2.unstack(1).stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [23]:
df = pd.DataFrame({'left': result, 'right': result+5},
                  columns=pd.Index(['left', 'right'], name='side')
                 )
df

side             left  right
number state                
one    Ohio         0      5
       Colorado     3      8
two    Ohio         1      6
       Colorado     4      9
three  Ohio         2      7
       Colorado     5     10

In [24]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [25]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Timeseries by columns

This is an example of taking the columns "year" and "quarter" and turning them into a time series index. By reindexing the colums, we also throw out the columns we don't want to deal with and are left with only GDP, inflation, and unemployment numbers.

In [44]:
data = pd.read_csv('pydata-book/examples/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [39]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [45]:
columns = pd.Index(['realgdp', 'infl', 'unemp'])
columns

Index(['realgdp', 'infl', 'unemp'], dtype='object')

In [46]:
data = data.reindex(columns=columns)
data

,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2
...,...,...,...
198,13324.600,-3.16,6.0
199,13141.920,-8.79,6.9
200,12925.410,0.94,8.1
201,12901.504,3.37,9.2


In [49]:
data.index = periods.to_timestamp('D', 'end')
data

,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,13324.600,-3.16,6.0
2008-12-31 23:59:59.999999999,13141.920,-8.79,6.9
2009-03-31 23:59:59.999999999,12925.410,0.94,8.1


Below we re-arrange the data frame from "Wide" format, where GDP, inflation, and unemployment each have a column, to "Long" format, where there is one row per observation/measurement.

In [52]:
ldata = data.stack().reset_index()
ldata.columns = ['date', 'item', 'value']
ldata

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
...,...,...,...
604,2009-06-30 23:59:59.999999999,infl,3.370
605,2009-06-30 23:59:59.999999999,unemp,9.200
606,2009-09-30 23:59:59.999999999,realgdp,12990.341
607,2009-09-30 23:59:59.999999999,infl,3.560


### Pivoting "Long" to "Wide" Format
Data is often stored this way in relational databases, "TERM_FOR_LICENSE" is an example of this in the ERMS schema.  For this reason, pandas provide a simple pivot operation to pull out the item/value columns into a column for each item.

In [53]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


The arguments here are the row and column index, and then an optional column to be taken as the value for the pivoted data frame.  If you have multiple value columns you can skip this and end-up with a hierarchical column index.

In [54]:
ldata['value2'] = np.random.randn(len(ldata))
ldata

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-1.257205
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.815554
2,1959-03-31 23:59:59.999999999,unemp,5.800,0.017756
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-0.024722
4,1959-06-30 23:59:59.999999999,infl,2.340,-0.688455
...,...,...,...,...
604,2009-06-30 23:59:59.999999999,infl,3.370,1.262965
605,2009-06-30 23:59:59.999999999,unemp,9.200,-1.005111
606,2009-09-30 23:59:59.999999999,realgdp,12990.341,0.940886
607,2009-09-30 23:59:59.999999999,infl,3.560,0.879725


In [57]:
pivoted = ldata.pivot('date', 'item')
pivoted

value                     value2            \
item                           infl    realgdp unemp      infl   realgdp   
date                                                                       
1959-03-31 23:59:59.999999999  0.00   2710.349   5.8 -0.815554 -1.257205   
1959-06-30 23:59:59.999999999  2.34   2778.801   5.1 -0.688455 -0.024722   
1959-09-30 23:59:59.999999999  2.74   2775.488   5.3 -0.279551 -0.055404   
1959-12-31 23:59:59.999999999  0.27   2785.204   5.6 -1.027127  0.284844   
1960-03-31 23:59:59.999999999  2.31   2847.699   5.2 -0.254207 -0.289769   
...                             ...        ...   ...       ...       ...   
2008-09-30 23:59:59.999999999 -3.16  13324.600   6.0 -0.688108  0.037399   
2008-12-31 23:59:59.999999999 -8.79  13141.920   6.9  0.492825 -0.160689   
2009-03-31 23:59:59.999999999  0.94  12925.410   8.1  0.932451 -2.403312   
2009-06-30 23:59:59.999999999  3.37  12901.504   9.2  1.262965 -0.476405   
2009-09-30 23:59:59.999999999  3.56  12990.341   9.6  0.879725  0.940886   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  0.017756  
1959-06-30 23:59:59.999999999  1.333736  
1959-09-30 23:59:59.999999999 -0.280239  
1959-12-31 23:59:59.999999999  0.114676  
1960-03-31 23:59:59.999999999 -1.571883  
...                                 ...  
2008-09-30 23:59:59.999999999 -0.209017  
2008-12-31 23:59:59.999999999 -0.436486  
2009-03-31 23:59:59.999999999 -0.300330  
2009-06-30 23:59:59.999999999 -1.005111  
2009-09-30 23:59:59.999999999 -0.068238  

[203 rows x 6 columns]

In [58]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


Pivot is equivalent to creating an index by date and item, and then unstacking the item.

In [59]:
ldata.set_index(['date', 'item']).unstack('item')

value                     value2            \
item                           infl    realgdp unemp      infl   realgdp   
date                                                                       
1959-03-31 23:59:59.999999999  0.00   2710.349   5.8 -0.815554 -1.257205   
1959-06-30 23:59:59.999999999  2.34   2778.801   5.1 -0.688455 -0.024722   
1959-09-30 23:59:59.999999999  2.74   2775.488   5.3 -0.279551 -0.055404   
1959-12-31 23:59:59.999999999  0.27   2785.204   5.6 -1.027127  0.284844   
1960-03-31 23:59:59.999999999  2.31   2847.699   5.2 -0.254207 -0.289769   
...                             ...        ...   ...       ...       ...   
2008-09-30 23:59:59.999999999 -3.16  13324.600   6.0 -0.688108  0.037399   
2008-12-31 23:59:59.999999999 -8.79  13141.920   6.9  0.492825 -0.160689   
2009-03-31 23:59:59.999999999  0.94  12925.410   8.1  0.932451 -2.403312   
2009-06-30 23:59:59.999999999  3.37  12901.504   9.2  1.262965 -0.476405   
2009-09-30 23:59:59.999999999  3.56  12990.341   9.6  0.879725  0.940886   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  0.017756  
1959-06-30 23:59:59.999999999  1.333736  
1959-09-30 23:59:59.999999999 -0.280239  
1959-12-31 23:59:59.999999999  0.114676  
1960-03-31 23:59:59.999999999 -1.571883  
...                                 ...  
2008-09-30 23:59:59.999999999 -0.209017  
2008-12-31 23:59:59.999999999 -0.436486  
2009-03-31 23:59:59.999999999 -0.300330  
2009-06-30 23:59:59.999999999 -1.005111  
2009-09-30 23:59:59.999999999 -0.068238  

[203 rows x 6 columns]

### Pivoting "Wide" to "Long" format
The dataframe method "melt" goes the reverse direction, producing a longer DataFrame.

In [60]:
df = pd.DataFrame({
    'key': ['foo', 'bar', 'baz'],
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [62]:
melted = df.melt('key')
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [64]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [69]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


When "melting" a dataframe, you don't have to take all observations as variables to be used in the long column, you may pick columns.

In [66]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [67]:
df.melt('key', value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
